In [43]:
from keras.models import Model
from keras.layers import Input, Conv2D, Activation, BatchNormalization, Dropout, concatenate, Conv2DTranspose
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [54]:
def load_images():
    Scale = 0.0
    with open(os.path.join('Beleg', 'Scale.txt'), 'r') as f:
        Scale = float(f.readline())
        
    path_train_color = os.path.join('Beleg', 'train', 'Color')
    path_train_ir = os.path.join('Beleg', 'train', 'IR')
    path_train_depth = os.path.join('Beleg', 'train', 'Depth')
    
    path_test_color = os.path.join('Beleg', 'test', 'Color')
    path_test_ir = os.path.join('Beleg', 'test', 'IR')
    path_test_depth = os.path.join('Beleg', 'test', 'Depth')
    
    files_train_color = os.listdir(path_train_color)
    files_test_color = os.listdir(path_test_color)
    
    train_color = [file for file in files_train_color if file[-4:] == '.jpg']
    test_color = [file for file in files_test_color if file[-4:] == '.jpg']
    
    X_train_color = []
    X_train_ir = []
    y_train = []
    X_test_color = []
    X_test_ir = []
    y_test = []
    invalid_images = 0
    
    for file in train_color:
        color = cv2.imread(os.path.join(path_train_color, file))
        ir = cv2.imread(os.path.join(path_train_ir, file), cv2.IMREAD_GRAYSCALE)
        depth = cv2.imread(os.path.join(path_train_depth, file[:-4]+'.png'), cv2.IMREAD_GRAYSCALE)
        if color is None or ir is None or depth is None:
            invalid_images += 1
            continue
        color = cv2.resize(color, (256,256), interpolation=cv2.INTER_AREA)
        ir = cv2.resize(ir, (256,256), interpolation=cv2.INTER_AREA)
        depth = cv2.resize(depth, (256,256), interpolation=cv2.INTER_AREA)
        #depth = np.asarray(Scale*depth, dtype=np.float32)
        X_train_color.append(color)
        X_train_ir.append(ir)
        y_train.append(depth)
        
    for file in test_color:
        color = cv2.imread(os.path.join(path_test_color, file))
        ir = cv2.imread(os.path.join(path_test_ir, file), cv2.IMREAD_GRAYSCALE)
        depth = cv2.imread(os.path.join(path_test_depth, file[:-4]+'.png'), cv2.IMREAD_GRAYSCALE)
        if color is None or ir is None or depth is None:
            invalid_images += 1
            continue
        color = cv2.resize(color, (256,256), cv2.INTER_AREA)
        ir = cv2.resize(ir, (256,256), cv2.INTER_AREA)
        depth = cv2.resize(depth, (256,256), cv2.INTER_AREA)
        #depth = np.asarray(Scale*depth, dtype=np.float32)
        X_test_color.append(color)
        X_test_ir.append(ir)
        y_test.append(depth)
        
    return np.asarray(X_train_color), np.asarray(X_train_ir), np.asarray(y_train), np.asarray(X_test_color), np.asarray(X_test_ir), np.asarray(y_test)

In [55]:
X_train_color, X_train_ir, y_train, X_test_color, X_test_ir, y_test = load_images()

In [56]:
input_color = Input(shape=(256,256,3), name="Color_Input")
input_ir = Input(shape=(256,256,1), name="IR_Input")

# Color branch
x = Conv2D(256, (5,5), strides=(2,2), padding="same", activation="relu", name="Color_Conv2D_1")(input_color)
x = Conv2D(128, (3,3), strides=(2,2), padding="same", activation="relu", name="Color_Conv2D_2")(x)
x = Model(inputs=input_color, outputs=x)

# IR branch
y = Conv2D(256, (5,5), strides=(2,2), padding="same", activation="relu", name="IR_Conv2D_1")(input_ir)
y = Conv2D(128, (3,3), strides=(2,2), padding="same", activation="relu", name="IR_Conv2D_2")(y)
y = Model(inputs=input_ir, outputs=y)

# combine both branches
combined = concatenate([x.output, y.output], name="Concat")

z = Conv2D(64, (3,3), strides=(1,1), padding="same", activation="relu", name="Comb_Conv2D_1")(combined)
z = Conv2DTranspose(128, kernel_size=(3,3), strides=(2,2), padding="same", activation="relu", name="Comb_DeConv2D_1")(z)
z = Conv2DTranspose(256, kernel_size=(5,5), strides=(2,2), padding="same", activation="relu", name="Comb_DeConv2D_2")(z)
z = Conv2D(256, kernel_size=(5,5), strides=(1,1), padding="same", activation="relu", name="Comb_Conv2D_2")(z)
z = Conv2D(1, kernel_size=(3,3), strides=(1,1), padding="same", activation="relu", name="Comb_Conv2D_3")(z)

model = Model(inputs=[x.input, y.input], outputs=z)

model.compile(
    optimizer="adam",
    loss="mae",
    metrics=['mae', 'mse'])

In [57]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Color_Input (InputLayer)        (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
IR_Input (InputLayer)           (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
Color_Conv2D_1 (Conv2D)         (None, 128, 128, 256 19456       Color_Input[0][0]                
__________________________________________________________________________________________________
IR_Conv2D_1 (Conv2D)            (None, 128, 128, 256 6656        IR_Input[0][0]                   
__________________________________________________________________________________________________
Color_Conv

In [58]:
model.fit(
    [X_train_color, X_train_ir.reshape(-1,256,256,1)],
    y_train.reshape(-1, 256,256,1),
    epochs=10,
    batch_size = 10,
    validation_split=0.2)

Train on 159 samples, validate on 40 samples
Epoch 1/10
159/159 [==============================] - 8s 52ms/step - loss: 20.2807 - mean_absolute_error: 20.2807 - mean_squared_error: 1325.2916 - val_loss: 7.4418 - val_mean_absolute_error: 7.4418 - val_mean_squared_error: 80.8077
Epoch 2/10
159/159 [==============================] - 7s 47ms/step - loss: 5.0790 - mean_absolute_error: 5.0790 - mean_squared_error: 43.7943 - val_loss: 3.8825 - val_mean_absolute_error: 3.8825 - val_mean_squared_error: 25.8778
Epoch 3/10
159/159 [==============================] - 7s 47ms/step - loss: 4.0309 - mean_absolute_error: 4.0309 - mean_squared_error: 30.6587 - val_loss: 3.5204 - val_mean_absolute_error: 3.5204 - val_mean_squared_error: 21.7867
Epoch 4/10
159/159 [==============================] - 7s 47ms/step - loss: 3.9543 - mean_absolute_error: 3.9543 - mean_squared_error: 29.9228 - val_loss: 3.4697 - val_mean_absolute_error: 3.4697 - val_mean_squared_error: 20.8659
Epoch 5/10
159/159 [===============

In [59]:
print(y_test[0])

[[ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 4  3 11 ...  0  0  0]
 [ 2  8 11 ...  0  0  0]
 [ 0 11 11 ...  0  0  0]]


In [60]:
d = model.predict([X_test_color[0].reshape(1,256,256,3), X_test_ir[0].reshape(-1, 256,256,1)])

In [62]:
print(d.shape
     )

(1, 256, 256, 1)


In [63]:
d.reshape(256,256)

array([[1.1790194e-02, 1.1515117e+00, 2.9213803e+00, ..., 1.9578234e+01,
        1.5119235e+01, 8.5555696e+00],
       [0.0000000e+00, 2.3562118e-01, 1.1595874e+00, ..., 1.8867699e+01,
        1.6861765e+01, 6.0285015e+00],
       [0.0000000e+00, 0.0000000e+00, 5.6677002e-01, ..., 1.8725613e+01,
        1.5308620e+01, 7.8550739e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 8.2696228e+00,
        6.8757601e+00, 2.9774649e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 7.5296264e+00,
        5.5849175e+00, 2.7945225e+00],
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 6.7869473e+00,
        4.9226336e+00, 2.2981105e+00]], dtype=float32)